In [24]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
sound_file = 'C:/Users/eduds/Music/jupyther_ring.wav' #sonido para que avise cuando termine de ejecutar proceso

In [22]:
#Borrar Espectrogramas
import shutil
import os

for folder in os.listdir('C:/Users/eduds/Desktop/Sample Class/Spectrograms'):
    shutil.rmtree('C:/Users/eduds/Desktop/Sample Class/Spectrograms/' + folder)
    os.makedirs('C:/Users/eduds/Desktop/Sample Class/Spectrograms/' + folder)

In [ ]:
import librosa
import librosa.display
import pylab
#Crear espectrogramas y exportarlos en png
for clase in os.listdir('C:/Users/eduds/Desktop/Sample Class/Audios'):
    #Kicks, Snares, Claps, Hats
    a = os.listdir('C:/Users/eduds/Desktop/Sample Class/Audios/' + clase) #obtiene lista de archivos
    
    
    
    
    for filename in a: #Filenames
        audio, sr = librosa.load('C:/Users/eduds/Desktop/Sample Class/Audios/'+ clase + '/' + filename)
        
        audio = librosa.util.normalize(audio) #Normalizado
        
        audio_trim, index = librosa.effects.trim(audio, top_db=40)
        audio_cut = librosa.util.fix_length(audio_trim, 3000)
    
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) #que hace esto
        
        S = librosa.feature.melspectrogram(y=audio_cut, power=0.5, n_mels = 256, hop_length = 5)
        librosa.display.specshow(S, x_axis='time', y_axis='log')

        pylab.savefig('C:/Users/eduds/Desktop/Sample Class/Spectrograms/' + clase + '/' + filename[:-4] +'.png', bbox_inches=None, pad_inches=0)
        pylab.close()
        
Audio(sound_file, autoplay=True, rate=44100) #te avisa con un sonido cuando termina de ejecutar

In [66]:
from sklearn.utils import Bunch
from skimage.io import imread
from skimage.transform import resize

def load_image_files(container_path, dimension=(64, 64)):
    
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []
    
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = imread(file)
            img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(data = flat_data,
                 target = target,
                 target_names = categories,
                 images = images,
                 DESCR = descr)

In [67]:
image_dataset = load_image_files("C:/Users/eduds/Desktop/Sample Class/Spectrograms")

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.3, random_state=109)

In [70]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm, metrics, datasets
import time
start = time.time()

param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

svc = svm.SVC()
clf = GridSearchCV(svc, param_grid)
clf.fit(X_train, y_train)

end = time.time()
print(end-start)

111.35131287574768


In [91]:
y_pred = clf.predict(X_test)

In [99]:
X_test[0].shape

(16384,)

In [73]:
print("Classification report for - \n{}:\n{}\n".format(
    clf, metrics.classification_report(y_test, y_pred)))

Classification report for - 
GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0):
              precision    recall  f1-score   support

           0       0.89      0.94      0.91        50
           1       0.97      0.92      0.94        63
           2   

# Predicciones individuales de audios

In [ ]:
#Primero conviertes el audio a spectrograma

file_name = 'Snare Drum 027' #solo poner aqui el nombre de cualquier archivo de audio wav

file_path = 'C:/Users/eduds/Desktop/Sample Class/Audio Prueba/' #directorio donde guardar los audios que quieres predecir

audio, sr = librosa.load(file_path + file_name + '.wav')
        
audio = librosa.util.normalize(audio) #Normalizado

audio_trim, index = librosa.effects.trim(audio, top_db=40)
audio_cut = librosa.util.fix_length(audio_trim, 3000)

pylab.axis('off') # no axis
pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) #que hace esto

S = librosa.feature.melspectrogram(y=audio_cut, power=0.5, n_mels = 256, hop_length = 5)
librosa.display.specshow(S, x_axis='time', y_axis='log')

# directorio donde se guardaran los spectrogramas
pylab.savefig('C:/Users/eduds/Desktop/Sample Class/Spectrograma Prueba/'+ file_name + '.png', bbox_inches=None, pad_inches=0)
pylab.close()

In [131]:
#introducir el nombre del archivo a producir una vez creado su spectrograma
archivo_predecir = 'Clap 001 808 Bright'

#Pasa por el mismo proceso que las imagenes de entrenamiento del modelo
img = imread('C:/Users/eduds/Desktop/Sample Class/Spectrograma Prueba/' + archivo_predecir + '.png')
img_resized = resize(img, (64, 64), anti_aliasing=True, mode='reflect')
img_resized = img_resized.flatten()
flat_data = np.array(img_resized)
a = flat_data.reshape(1, -1)

#Prediccion de clase
y_pred = clf.predict(a)
y_pred

array([0])

In [ ]:
'''
Clases
0 = Claps
1 = Hats
2 = Kicks
3 = Snare
'''